In [3]:
pip install python-weka-wrapper3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 67.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 27.6 MB/s eta 0:00:00
  Created wheel for python-weka-wrapper3: filename=python_weka_wrapper3-0.3.2-py3-none-any.whl size=14495848 sha256=83a2f42763bbdfc99aee69082df1eebc70289fc481903bb78db35b34f5ae6e8c
  Stored in directory: /root/.cache/pip/wheels/67/6c/8b/ac3ddbec2fff91bdbedc52d8f6bdc2199bcec5338002f164ed
  Created wheel for configurable-objects: filename=configurable_objects-0.0.1-py3-none-any.whl size=4696 sha256=0a145c229f073367f0ce1b06e71c8199c202682118eb9638d2ff1fd247d23475
  Stored in directory: /root/.cache/pip/wheels/ef/11/bc/75ac8b0592c38dc42412942c37d3947faf0b222bad150132a1
  Created wheel for simple-data-flow: filename=simple_data_flow-0.0.1-py3-none-any.whl size=19058 sha2

In [15]:
import pickle
import weka.core.jvm as jvm
import weka.core.converters as converters
import weka.classifiers as classifiers
import weka.core.dataset as dataset

In [3]:
# Iniciar a JVM do Weka
jvm.start()

# Carregar os dados do .csv
df = pd.read_csv("/content/risco_credito.csv")

# Mapear valores categóricos para números inteiros
historia_map = {"ruim": 0, "desconhecida": 1, "boa": 2}
divida_map = {"alta": 0, "baixa": 1}
garantias_map = {"nenhuma": 0, "adequada": 1}
renda_map = {"0_15": 0, "15_35": 1, "acima_35": 2}
risco_map = {"baixo": 0, "moderado": 1, "alto": 2}

df["historia"] = df["historia"].map(historia_map)
df["divida"] = df["divida"].map(divida_map)
df["garantias"] = df["garantias"].map(garantias_map)
df["renda"] = df["renda"].map(renda_map)
df["risco"] = df["risco"].map(risco_map)

# Converter os dados para o formato do Weka
attributes = [
    dataset.Attribute.create_numeric("historia"),
    dataset.Attribute.create_numeric("divida"),
    dataset.Attribute.create_numeric("garantias"),
    dataset.Attribute.create_numeric("renda")
]
class_attribute = dataset.Attribute.create_nominal("risco", ["baixo", "moderado", "alto"])

# Criar o conjunto de dados
data = dataset.Instances.create_instances("risco_credito", attributes + [class_attribute], 0)

# Adicionar instâncias ao conjunto de dados
for index, row in df.iterrows():
    inst_values = [
        float(row["historia"]), float(row["divida"]), float(row["garantias"]),
        float(row["renda"]), row["risco"]
    ]
    instance = dataset.Instance.create_instance(inst_values)
    data.add_instance(instance)

data.class_is_last()

# Configurar o classificador OneR
classifier = classifiers.Classifier(classname="weka.classifiers.rules.OneR")
classifier.build_classifier(data)

# Exibir o resultado
print(classifier)

# Parar a JVM do Weka
jvm.stop()


INFO:weka.core.jvm:JVM already running, call jvm.stop() first


garantias:
	< 0.5	-> alto
	>= 0.5	-> baixo
(8/14 instances correct)



In [2]:
import pandas as pd

# Carregar os dados do .csv
df = pd.read_csv("/content/risco_credito.csv")

# Exibir a relação entre 'renda' e 'risco'
print(df.groupby('renda')['risco'].value_counts())

# Calcular a taxa de risco para cada nível de renda
renda_risco = df.groupby(['renda', 'risco']).size().unstack(fill_value=0)
renda_risco['total'] = renda_risco.sum(axis=1)
renda_risco['risco_alto_ratio'] = renda_risco['alto'] / renda_risco['total']
print(renda_risco)


renda     risco   
0_15      alto        3
15_35     alto        2
          moderado    2
acima_35  baixo       5
          alto        1
          moderado    1
Name: count, dtype: int64
risco     alto  baixo  moderado  total  risco_alto_ratio
renda                                                   
0_15         3      0         0      3          1.000000
15_35        2      0         2      4          0.500000
acima_35     1      5         1      7          0.142857
